In [ ]:
# default_exp facade


In [ ]:
#export
import random
from nbdev.showdoc import *
import pandas as pd
import sentencepiece as sp
from pathlib import Path
from collections import Counter
from ds4se.mining.unsupervised.traceability.eval import *
#import ds4se.mining.unsupervised.traceability.approach.cisco as cisco
from enum import Enum, unique, auto
from ds4se.exp import i
import os
import pkg_resources
from sklearn.manifold import TSNE
import numpy as np

In [ ]:
#export
@unique
class LinkType(Enum):
    req2tc = auto()
    req2src = auto()

In [ ]:
#Helper functions for data analysis part of library

#export
def get_docs(df, spm):
    #param dataframe of content that need to be processed
    #param sentence piece processor that will process the dataframe
    #return the document list
    docs = []
    for fn in df["contents"]:
        docs += spm.EncodeAsPieces(fn)
    return docs

#export
def get_counters(docs):
    #param doc list of contents that need info on tokens
    #return the counters object of tokens   
    doc_cnts = []
    cnt = Counter()
    for tok in docs:
        cnt[tok] += 1 
        doc_cnts.append(cnt)
    return doc_cnts

#export
#load sentence piece model and call two helper function to calculate token freqnency 
def preprocess(artifacts_df):
    spm = sp.SentencePieceProcessor()
    bpe_model_path = pkg_resources.resource_filename('ds4se', 'model/test.model')
    spm.Load(bpe_model_path)
    docs = get_docs(artifacts_df,spm)
    cnts = get_counters(docs)
    return cnts

In [ ]:
#export
def TraceLinkValue(source, target, technique, word2vec_metric = "WMD"):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #param technique what tecchnique to use to calculate trace values
    #return a tuple: (distance, similarity)
    
    #TODO make the string saved and then return the path to that string
    sourcePath = "path/to/something"#this is where we save the passed in source
    targetPath = "./something"
    
    dummy_path = pkg_resources.resource_filename('ds4se', 'model/val.csv')

    
    value = random.randint(0,1)/100

    
    if (technique == "VSM"):
        pass
    if (technique == "LDA"):
        pass
    if (technique == "orthogonal"):
        pass
    if (technique == "LSA"):
        pass
    if (technique == "JS"):
        pass
    if (technique == "word2vec"):
        model_path = pkg_resources.resource_filename('ds4se', 'model/word2vec_libest.model')
        parameter = {
            "vectorizationType": VectorizationType.word2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": model_path,
            "source_path": dummy_path,
            "target_path": dummy_path,
            "system_path": dummy_path,
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        
        source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
        target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
        word2vec = Word2VecSeqVect(parameter)
        word2vec.df_source = source_df
        word2vec.df_target = target_df
        links = [(source_df["ids"][0],target_df["ids"][0])]
        if (word2vec_metric == "SCM"):
            computeDistanceMetric = word2vec.computeDistanceMetric(links, metric_list = [DistanceMetric.SCM])
        else:
            computeDistanceMetric = word2vec.computeDistanceMetric(links, metric_list = [DistanceMetric.WMD])
        value = (computeDistanceMetric[0][0][2],computeDistanceMetric[0][0][3])    
        
    if (technique == "doc2vec"):
        model_path = pkg_resources.resource_filename('ds4se', 'model/doc2vec_libest.model')
        parameter = {
            "vectorizationType": VectorizationType.doc2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": model_path,
            "source_path": dummy_path,
            "target_path": dummy_path,
            "system_path": dummy_path,
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        
        source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
        target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
        doc2vec = Doc2VecSeqVect(params = parameter)
        doc2vec.df_source = source_df
        doc2vec.df_target = target_df
        links = [(source_df["ids"][0],target_df["ids"][0])]
        doc2vec.InferDoc2Vec(steps=200)
        table = doc2vec.computeDistanceMetric( links, metric_list = [DistanceMetric.EUC] )
        value = (table[0][0][2], table[0][0][3])
        #The bottom is here for reference -- may not need it
#         doc2vec.SaveLinks()
#         #will most likely need to change this part need to change this part to a different path
#         path_to_ground_truth = '/tf/main/benchmarking/traceability/testbeds/groundtruth/english/[libest-ground-req-to-tc].txt'
#         doc2vec.MatchWithGroundTruth(path_to_ground_truth)
#         doc2vec.SaveLinks(grtruth = True)
#         #TODO find logic to LoadLink properly and display what is needed
    
    return value

In [ ]:
str1 = open('test_data/RQ11.txt', 'r').read()
str2 = open('test_data/us4020.c', 'r').read()
TraceLinkValue(str1,str2,"doc2vec")


source_file = pd.read_csv("test_data/[libest-pre-req].csv",names=['ids', 'text'], header=None, sep=' ')
target_file = pd.read_csv("test_data/[libest-pre-tc].csv",names=['ids', 'text'], header=None, sep=' ')
source = source_file['text'][4]  
target = target_file['text'][4]
result = TraceLinkValue(target, source, "word2vec")    
print(result)

2020-11-09 09:21:36,582 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-11-09 09:21:36,592 : INFO : built Dictionary(1815 unique tokens: ['@return', 'Converts', 'The', 'a', 'and']...) from 153 documents (total 5769 corpus positions)
2020-11-09 09:21:36,593 : INFO : loading Doc2Vec object from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model
2020-11-09 09:21:36,613 : INFO : loading vocabulary recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model.vocabulary.* with mmap=None
2020-11-09 09:21:36,613 : INFO : loading trainables recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model.trainables.* with mmap=None
2020-11-09 09:21:36,614 : INFO : loading wv recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model.wv.* with mmap=None
2020-11-09 09:21:36,615 : INFO : loading docvecs recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_lib

(0.4803363918627718, 0.6755221350342244)


In [ ]:
expected = (0.5401013460499512, 0.6493079189657213)

In [ ]:
#FacadeTest
#Test case for proto TLV

#Prototype should print a value between 0 or 1. Input values aren't used

#test doc2vec
str1 = open('test_data/RQ11.txt', 'r').read()
str2 = open('test_data/us4020.c', 'r').read()
testTLV = TraceLinkValue(str1,str2,"doc2vec")
assert(isinstance(testTLV,tuple))
assert(testTLV == (31.28299903869629, 0.030976056431477807))

#test word2vec
source_file = pd.read_csv("test_data/[libest-pre-req].csv",names=['ids', 'text'], header=None, sep=' ')
target_file = pd.read_csv("test_data/[libest-pre-tc].csv",names=['ids', 'text'], header=None, sep=' ')
source = source_file['text'][4]  
target = target_file['text'][4]
result = TraceLinkValue(target, source, "word2vec")    
assert(result == (0.4803363918627718, 0.6755221350342244))

2020-11-09 09:22:21,934 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-11-09 09:22:21,943 : INFO : built Dictionary(1815 unique tokens: ['@return', 'Converts', 'The', 'a', 'and']...) from 153 documents (total 5769 corpus positions)
2020-11-09 09:22:21,944 : INFO : loading Doc2Vec object from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model
2020-11-09 09:22:21,963 : INFO : loading vocabulary recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model.vocabulary.* with mmap=None
2020-11-09 09:22:21,963 : INFO : loading trainables recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model.trainables.* with mmap=None
2020-11-09 09:22:21,964 : INFO : loading wv recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_libest.model.wv.* with mmap=None
2020-11-09 09:22:21,964 : INFO : loading docvecs recursively from /Users/yangchen/Desktop/CSCI435/ds4se/ds4se/model/doc2vec_lib

In [ ]:
#export
def NumDoc(source, target):
    #param source a dataframe of the entire source file  
    #param target a dataframe of the entire target file 
    #return a list containing the the difference between the two files
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    difference = source_doc - target_doc
    return [source_doc, target_doc, difference, -difference]

[3, 3, 0, 0]

In [ ]:
#FacadeTest
#Test Case for Proto NumDoc

#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
testND = NumDoc(df, df2)
expected = [3,3,0,0]
assert(isinstance(testND, list))
assert(testND[0] == expected[0])
assert(testND[1] == expected[1])
assert(testND[2] == expected[2])
assert(testND[3] == expected[3])

In [ ]:
#export
def VocabSize(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return a list containing the the difference between the two files in terms of vocab
    source_list = preprocess(source)
    target_list = preprocess(target)
    source_size = len(source_list[0])
    target_size = len(target_list[0])
    difference = source_size - target_size
    return [source_size, target_size, difference, -difference]  

In [ ]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
VocabSize(df, df2)

[17, 18, -1, 1]

In [ ]:
#FacadeTest
#Test case for Vocab size
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
VocabSize(df, df2)
#Prototype should return an array based on random number values. Inputs aren't used.
testVS = VocabSize(df1,df2)
expected = [17, 18, -1, 1]
assert(testVS[0] == expected[0])
assert(testVS[1] == expected[1])
assert(testVS[2] == expected[2])
assert(testVS[3] == expected[3])

In [ ]:
#export
def AverageToken(source, target):
    #param source a dataframe of the entire source file  
    #param target a dataframe of the entire target file 
    #return a list containing the the difference between the two files in terms of tokens
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    
    source_list = preprocess(source)
    target_list = preprocess(target)
    
    source_total_token = sum(source_list[0].values())
    target_total_token = sum(target_list[0].values())

    source_token = source_total_token/source_doc
    target_token = target_total_token/target_doc
    difference = source_token - target_token
    return [source_token, target_token, difference, -difference]

In [ ]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
AverageToken(df,df2)

[15.333333333333334, 16.0, -0.6666666666666661, 0.6666666666666661]

In [ ]:
#FacadeTest
#Test Case for AverageToken

d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
test = AverageToken(df,df2)
assert(test == [15.333333333333334, 16.0, -0.6666666666666661, 0.6666666666666661])

In [ ]:
#export
def Vocab(artifacts_df):
    #we can add a parameter for user to specify the number of most frequent token to return
    #param a dataframe of contents that need to be processed
    #return a list containing the the difference between the two files in terms of vocab
    cnts = preprocess(artifacts_df)
    vocab_list = cnts[0].most_common(3)
    total = sum(cnts[0].values())
    vocab_dict = dict()
    vocab_dict[vocab_list[0][0]] = [vocab_list[0][1], vocab_list[0][1]/total]
    vocab_dict[vocab_list[1][0]] = [vocab_list[1][1], vocab_list[1][1]/total]
    vocab_dict[vocab_list[2][0]] = [vocab_list[2][1], vocab_list[2][1]/total]

    return vocab_dict

In [ ]:
#FacadeTest
#Test Case for Vocab

#Prototype Vocab should return dictionary type. 
#Will expand to test dictionary values once presets can be generated and tested
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
testVocab = Vocab(df)
assert(isinstance(testVocab,dict))

assert(testVocab == {'▁': [23, 0.5],
 'this': [2, 0.043478260869565216],
 'is': [2, 0.043478260869565216]})


In [ ]:
  #export
def VocabShared(source, target):
    #param source a dataframe of the entire source file  
    #param target a dataframe of the entire target file 
    #return the similarities of vocab in the files 
    df = pd.concat([source, target])
    return Vocab(df) 

In [ ]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana banana banana banana banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
VocabShared(df, df2)

{'▁': [52, 0.5],
 'this': [5, 0.04807692307692308],
 'banana': [5, 0.04807692307692308]}

In [ ]:
#FacadeTest
#Test VocabShared

#Prototype should show a dictionary
#Test will be expanded to verify frequency verification in two samples
d = {'contents': ['this is a content of the source file', 'hello world', 'this this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana banana banana banana banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
testVocabS = VocabShared(df, df2)
assert(isinstance(testVocabS,dict))
assert(testVocabS == {'▁': [52, 0.5],
 'this': [5, 0.04807692307692308],
 'banana': [5, 0.04807692307692308]})


In [ ]:
#export
def SharedVocabSize(source, target):
    #param source a dataframe of the entire source file  
    #param target a dataframe of the entire target file 
    #return the similarities of vocab sizes in the files 
    df = pd.concat([source, target])
    df_counts = preprocess(df)
    shared_size = len(df_counts[0])
    return shared_size

In [ ]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
SharedVocabSize(df, df)

17

In [ ]:
#FacadeTest
#Test Shared vocab size

#Prototype should return an int
d = {'contents': ['this is a content of the source file', 'hello world', 'this this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)

testSVS = SharedVocabSize(df, df)
assert(testSVS)
assert(testSVS == 17)

In [ ]:
################## Dummy Tests

In [ ]:
#export
def MutualInformation(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the mutual information 
    mutual_information = random.randint(100,200)
    return mutual_information

In [ ]:
#FacadeTest
#Test Mutual information

#Proto mutual information should get an int
testMutualInfo = MutualInformation("source","target")
assert(testMutualInfo in range(100,200))

In [ ]:

df2 = pd.DataFrame(data=target)
i.dit_shannon(preprocess(df)[0])
df

,contents
0,this is a content of the source file
1,hello world
2,this this is the content of another source fil...


In [ ]:
#export
def CrossEntropy(source, target):
    #param source a dataframe of the entire source artifact  
    #param target a dataframe of the entire target artifact 
    #return the entropy
    combined = source.append(target)
    entropy = i.dit_shannon(preprocess(combined)[0])

    return entropy
    

#     cross_entropy = random.randint(100,200)#looks like it is the msi funciton in the classes for word2vec or doc2vec
#     cross_entropy = get_system_entropy_from_df(source, "col1",)
#     return cross_entropy

In [ ]:
#FacadeTest
#Test Cross Entropy

#Proto cross Entropy should return an int
testCrossEntropy = CrossEntropy("Sauce","Target")
assert(testCrossEntropy in range(100,200))

#Eventually cross entropy will properly implement Entropy calculation and the test will compare a known/predicted entropy
# to a calculated value


AttributeError: 'str' object has no attribute 'append'

In [ ]:
def makeArray(text):
    print("text:",text)
    return np.fromstring(text[1:-1],sep=' ')
    
#export
def KLDivergence(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the divergence  
    
    #we are going to use the TSNE function since this preforms the divergence
    source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
    target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
    source_df = source_df["text"].apply(makeArray)
    target_ef = target_df["text"].apply(makeArray)
    source_df.head()
    target_df.head()
    divergence = random.randint(100,200)
    return divergence

In [ ]:
#FacadeTest
#Test KLDivergence

#Proto KLDivegence will return an random int
testKLDiver = KLDivergence("source","target")
assert(testKLDiver in range(100,200))

text: source
text: target


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
